
# Chatbot IA avec Rag

Ce notebook implémente un chatbot IA avancé en utilisant **LangChain** et **LangGraph**.  
Il intègre :
- **RAG (Retrieval-Augmented Generation) avec affichage des sources**
- **Streaming des réponses pour une meilleure expérience utilisateur**
- **Mémoire conversationnelle pour un suivi des interactions**


import requests
 
ngrok_url = "https://01c6-163-5-3-68.ngrok-free.app"
response = requests.get(f"{ngrok_url}/collections")
 
print("Status Code:", response.status_code)
print("Response:", response.text)

In [80]:

!pip install --upgrade --quiet langchain langchain-community langchainhub beautifulsoup4 langchain-openai beautifulsoup4 chromadb
!pip install --quiet langgraph
!pip install --upgrade langchain-mistralai



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip


##  Importation des bibliothèques

In [1]:
# Importation des bibliothèques

import os
import json
import getpass

from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.prompts import PipelinePromptTemplate
from langchain_core.runnables import RunnableConfig


## Connexion à l'Api 

In [2]:
# Configuration API Mistral AI

MISTRAL_API_KEY = getpass.getpass("Entrez votre clé API Mistral: ")  
MODEL_NAME = "mistral-small-latest" #Modèle small pour combiner performance et prix accessible


In [3]:
# Initialisation du modèle / activation du streaming
llm = ChatMistralAI(
    mistral_api_key=MISTRAL_API_KEY,
    model=MODEL_NAME,
    streaming=True,
    temperature=0.7
)

##  Définition du Prompt

In [4]:
# Initialisation de la mémoire

memory = MemorySaver()

In [5]:
# Définition du Prompt Principal

prompt_template = ChatPromptTemplate.from_template(
    """
    Tu es **crystal bot** un **assistant d'orientation professionnelle**.
      
    Ton rôle est d'**aider l'utilisateur à faire des choix de carrière ou de formation** en fonction de **son parcours, sa personnalité, ses compétences et ses préférences**.  
    Ton objectif est d'**aider chaque utilisateur à clarifier son avenir** en lui proposant **des conseils sur-mesure, inspirants et bienveillants**.  

    ➤ **Adapte-toi** à l’utilisateur
    ➤ Si l’utilisateur semble **indécis ou curieux**, **propose des pistes d’exploration** (salons, témoignages, mentors, forums spécialisés).  
    ➤ **Si la personnalité est mentionnée mais non connue**, **invite subtilement à découvrir son MBTI** sans forcer.  
    ➤ **Si l’utilisateur cherche à comparer plusieurs options**, **aide-le à peser le pour et le contre** avec des arguments personnalisés.  
    ➤ Ajoute parfois une touche légère de mystère ou de vision d’avenir, sans trop en faire.  

    **Fournis une réponse détaillée, et motivante.**  
    Sois précis mais laisse de la place à l’exploration !  
    """
)

In [6]:
# Détection de l’intention utilisateur

intent_prompt = PromptTemplate.from_template(
    """
    Analyse la question suivante et identifie l’intention principale de l’utilisateur :

    Question: {input}

    Réponds par l’une ou plusieurs des options suivantes :
    - "Métiers"
    - "Formations"
    - "Salaire"
    - "Offres d'emploi"
    - "Parcoursup"
    - "Autre"

    Intention détectée :
    """
)


In [7]:
# Détection du domaine / topic

topic_prompt = PromptTemplate.from_template(
    """
    Analyse la question suivante et identifie le ou les domaines principaux auxquels elle appartient.

    Question: {input}

    Réponds en listant **un ou plusieurs domaines** parmi les suivants (séparés par une virgule) :
    - "Informatique et Numérique"
    - "Sciences et Ingénierie"
    - "Santé et Médecine"
    - "Droit et Sciences Politiques"
    - "Économie et Gestion"
    - "Sciences Sociales et Psychologie"
    - "Arts, Design et Culture"
    - "Lettres et Langues"
    - "Sciences de l'Éducation"
    - "Communication et Journalisme"
    - "Tourisme, Hôtellerie et Restauration"
    - "Environnement et Développement Durable"
    - "Sport et Activité Physique"
    - "Architecture et Urbanisme"
    - "Agronomie et Agroalimentaire"

    Si aucun domaine n’est clair, réponds uniquement par "Autre".

    Domaines identifiés :
    """
)


In [8]:
# Prompt réponse principale du chatbot

chatbot_prompt = PromptTemplate.from_template(
    """
    Je suis **Crystal Bot**, ton conseiller d'orientation professionnelle. 🔮✨  
    Mon objectif est de **t’aider à voir plus clair** dans ton avenir en te proposant des pistes adaptées et personnalisées.
    
    **Analyse bien l’utilisateur et adapte la réponse** :  
      
        - **S’il exprime une hésitation ou une peur**, encourage-le en mettant en avant plusieurs solutions rassurantes.  
        - **Si son profil semble en décalage avec ses aspirations**, donne des stratégies pour surmonter les obstacles (remises à niveau, formations adaptées).  
        - **Propose une réponse fluide et naturelle**, qui ne se contente pas d’une liste figée, mais qui établit un dialogue et donne une vision plus large.

    ** Intention de l'utilisateur : {intent}**  
    ** Profil de l’utilisateur** : {input}  
    ** Domaines détectés** : {topic}
    
    ➤ **Si l'intention est "Métiers"** :
        - Propose des **métiers adaptés** au profil de l’utilisateur en allant au-delà des évidences.   
        - **Donne une touche d’originalité** en citant des métiers émergents, des niches ou des perspectives d’évolution.
        - Explique **les missions, conditions de travail et débouchés** en t’adaptant à la curiosité de l’utilisateur.  
        - **N’hésite pas à raconter une anecdote ou une tendance actuelle, personnalise la réponse** pour rendre la réponse plus vivante et personnalisée.  
        - **Fournis une vision inspirante et dynamique**, et pas seulement une simple liste de métiers.

    ➤ **Si l'intention est "Formations"** :
        - Présente des **formations pertinentes** mais **ajoute des alternatives originales** (double cursus, parcours atypiques).
        - Décris les options de manière dynamique : **BTS, Licence, Master, écoles spécialisées, mais aussi MOOC et certifications**.  
        - **Varie les approches selon la question de l’utilisateur**, sans donner toujours la même liste figée.
        - Si pertinent, suggère des expériences internationales  ou des opportunités de spécialisation.  
        - Rassure l’utilisateur en expliquant que *sa motivation est un facteur clé*. *Donne lui des conseils*
        - **N’hésite pas à raconter une anecdote ou une tendance actuelle, personnalise la réponse** pour rendre la réponse plus vivante et personnalisée. 
        - Si pertinent propose à l'utilisateur de visiter des salons pour en savoir plus sur les formations. 

    ➤ **Si l'intention est "Salaire"** :
        - Donne **un aperçu détaillé des salaires**, mais va au-delà des chiffres standards en ajoutant **les tendances du marché**.  
        - Explique **les différences selon l’expérience, le secteur, et la localisation**.  
        - **Si pertinent, parle des avantages indirects** (avantages en nature, perspectives d’évolution, travail à l’international).  
         - **N’hésite pas à raconter une anecdote ou une tendance actuelle, personnalise la réponse** pour rendre la réponse plus vivante et personnalisée.  

    ➤ **Si l'intention est "Offres d'emploi"** :
        - **Décris le marché actuel** du secteur concerné : quelles entreprises recrutent ? Quelles compétences sont recherchées ?
        - Donne **des conseils personnalisés** pour optimiser les candidatures, en fonction du domaine visé.
        - **Propose une approche proactive** : comment créer des opportunités, réseauter efficacement ou se démarquer ?  
         - **N’hésite pas à raconter une anecdote ou une tendance actuelle, personnalise la réponse** pour rendre la réponse plus vivante et personnalisée.  

    ➤ **Si l'intention est "Parcoursup"** :
        - **Explique Parcoursup en t’adaptant au profil de l’utilisateur** (lycéen stressé, étudiant en réorientation…).
        - Fournis **des conseils concrets pour réussir son dossier** et éviter les pièges.  
        - Fournis **des conseils concrets pour réussir la lettre de motivation** et se distinguer des autres candidats. 
        - Si pertinent, parle **des alternatives à Parcoursup** (écoles privées, études à l’étranger, admissions parallèles).  
    

    ➤ **Si l'intention est "Autre"** :
        - **Interprète la demande de l’utilisateur avec souplesse** et propose une réponse originale.  
        - Si possible, **ouvre la discussion sur des idées nouvelles** en fonction de son profil.
        - **Demande lui plus de précisions sur ce qu'il souhaite apprendre ou approfondir** en terme professionnel ou de formation.
         
    """
)

In [9]:
# Conclusion dynamique

closing_prompt = PromptTemplate.from_template(
    """
    J’espère que cette réponse **t’a aidé à y voir plus clair !** 🔮  
    **As-tu d’autres questions ?** Je suis là pour explorer toutes les possibilités avec toi.  
    **Si tu hésites encore, n’hésite pas à me demander plus de précisions !**
    """
)


In [10]:
# Assemblage du pipeline 

pipeline_prompt = PipelinePromptTemplate(
    final_prompt=PromptTemplate.from_template(
        "{prompt_principal}\n\n{topic_detection}\n\n{intent_detection}\n\n{orientation_response}\n\n{closing}"
    ),
    pipeline_prompts=[
        ("prompt_principal", prompt_template),  
        ("topic_detection", topic_prompt),
        ("intent_detection", intent_prompt),
        ("orientation_response", chatbot_prompt),
        ("closing", closing_prompt), 
    ]
)

/var/folders/dh/4898rxls74sgb6h2nn4fnygh0000gn/T/ipykernel_84695/801868717.py:3: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  pipeline_prompt = PipelinePromptTemplate(


In [11]:
# Mise en place de la mémoire conversationnelle

memory = MemorySaver()  
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    """Appelle le modèle avec l'historique de conversation"""
    response = llm.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
chatbot = workflow.compile(checkpointer=memory)



In [15]:
# Fonction pour générer une réponse complète avec Crystal Bot
def generate_response(user_query: str):
    """ Génère une réponse optimisée avec Crystal Bot et conserve l’historique. """

    global conversation_history  # Utilisation d'une variable distincte pour stocker les messages

    # Charger l'historique des messages de manière locale
    past_messages = conversation_history if 'conversation_history' in globals() else []  

    # **🔹 Étape 1 : Détection du domaine / topic**
    topic_output = topic_prompt.format(input=user_query)

    # **🔹 Étape 2 : Détection de l’intention utilisateur**
    intent_output = intent_prompt.format(input=user_query)
    if not intent_output:
        intent_output = "Autre"

    # **🔹 Étape 3 : Génération de la réponse principale**
    chatbot_prompt_hidden = chatbot_prompt.format(
    input=user_query, topic=topic_output, intent=intent_output
    )

    chatbot_output = chatbot_prompt_hidden.replace(topic_output, "").replace(intent_output, "")

    # **🔹 Étape 4 : Génération des suggestions d’exploration selon l’intention détectée**
    exploration_suggestions = ""

    if "Métiers" in intent_output:
        exploration_suggestions += """
        **🌍 Pour aller plus loin :**  
        - 🔍 **Découvre des métiers similaires** via des plateformes comme Onisep ou Studyrama.  
        - 🎭 **Participe à des salons professionnels** et rencontres avec des experts du domaine.  
        - 👥 **Échange avec des professionnels** sur LinkedIn ou lors d’événements.  
        """

    if "Formations" in intent_output or "Parcoursup" in intent_output:
        exploration_suggestions += """
        **📚 Pour approfondir ton parcours :**  
        - 🎤 **Découvre les parcours d’anciens étudiants** via des témoignages en ligne.  
        - 🔎 **Explore les formations en alternance** et internationales.  
        - 🏫 **Assiste aux journées portes ouvertes** des écoles et universités.  
        """

    if "Salaire" in intent_output:
        exploration_suggestions += """
        **💰 Pour mieux comprendre les salaires et évolutions de carrière :**  
        - 📊 **Consulte des études de rémunération** sur Glassdoor et l’APEC.  
        - 📈 **Analyse les évolutions de carrière possibles** en fonction de ton secteur.  
        """

    if "Offres d'emploi" in intent_output:
        exploration_suggestions += """
        **📝 Pour trouver des opportunités professionnelles :**  
        - 🔍 **Consulte des plateformes spécialisées** comme Indeed ou Pôle Emploi.  
        - 📩 **Optimise ton CV et ta lettre de motivation** pour te démarquer.  
        """

    if "Les deux" in intent_output:
        exploration_suggestions += """
        **🌟 Pour lier formations et métiers :**  
        - 📚 **Découvre les formations qui recrutent le plus**.  
        - 👀 **Explore les tendances de recrutement dans ton secteur**.  
        """

    if "Autre" in intent_output:
        exploration_suggestions += """
        **🔍 Explorons d’autres pistes !**  
        - 🤔 **Précise un peu plus ta demande**, veux-tu parler de reconversion, d’entrepreneuriat, d’études à l’étranger ?  
        - 💡 **Inspiration** : Parfois, explorer d’autres secteurs peut ouvrir des portes inattendues.  
        - 🔎 **Découvre des parcours inspirants** : interviews, conférences, podcasts sur des choix de carrière atypiques.  
        """

    # **🔹 Étape 5 : Conclusion dynamique**
    closing_output = closing_prompt.format()

    # ✅ **Ajout d’une introduction engageante pour la toute première réponse**
    intro_message = f"""
    Bonjour ! 😊  

    Je suis **Crystal Bot**, ton conseiller d'orientation professionnelle. 🔮✨  
    Mon objectif est de **t’aider à clarifier ton avenir** en te proposant des pistes adaptées et personnalisées.  
    Voici quelques idées et conseils pour t’aider à avancer :  
    """

    # ✅ **Création du message final à envoyer au modèle**
    final_prompt = f"""
    {intro_message}  

    **💡 Recommandation de Crystal Bot :**  
    {chatbot_output}  

    {exploration_suggestions}  

    {closing_output}  
    """

    # ✅ **Ajout du message dans l'historique**
    messages = past_messages + [HumanMessage(content=final_prompt)]

    # ✅ **Envoi de la requête complète au modèle**
    response = llm.invoke(messages)

    # ✅ **Mise à jour de l'historique de la conversation**
    conversation_history = messages + [response]

    return {"response": response.content, "sources": ["Aucune source (VDB désactivée)"]}

##  Définition du Chatbot avec mémoire, RAG

## Test du Chatbot

In [13]:
# Test du chatbot avec une seule question utilisateur

user_input = "Je suis en terminal spécialité philosophie, mais en fait je préfère les sciences. Quelles études scientifiques pourrais-je faire malgré le fait que je n'ai pas de spé maths en terminal ?"
result = generate_response(user_input)

print("\n🔹 **Réponse du chatbot :**")
print(result["response"])



🔹 **Réponse du chatbot :**
### Intention détectée :
**Formations**

### Domaines identifiés :
**Sciences et Ingénierie**

---

Bonjour ! 😊

Je suis **Crystal Bot**, ton conseiller d'orientation professionnelle. 🔮✨

Ton intérêt pour les sciences est clairement indiqué, malgré le fait que tu n'aies pas choisi la spécialité maths en terminal. Pas de souci, il existe de nombreuses formations scientifiques accessibles même sans cette spécialité. Voici quelques pistes de formations qui pourraient t'intéresser :

1. **Licences Universitaires en Sciences** :
   - **Licence en Physique, Chimie ou Biologie** : Ces licences couvrent une large gamme de disciplines scientifiques et peuvent te permettre de te spécialiser par la suite.
   - **Licence en Sciences de la Terre et de l'Environnement** : Si tu es intéressé par les sciences de la Terre et l'environnement, cette formation pourrait être parfaite pour toi.

2. **BTS et DUT** :
   - **BTS en Biotechnologies** : Ce BTS te permet d'acquérir des

In [169]:
# Test après modification 

user_input = "Je suis en terminal spécialité philosophie, mais en fait je préfère les sciences. Quelles études scientifiques pourrais-je faire malgré le fait que je n'ai pas de spé maths en terminal ?"
result = generate_response(user_input)

print("\n🔹 **Réponse du chatbot :**")
print(result["response"])



🔹 **Réponse du chatbot :**
### **Recommandation de Crystal Bot :**

En tant que conseiller d'orientation professionnelle, je suis là pour t'aider à tracer ton chemin dans le monde des études scientifiques, même sans avoir choisi la spécialité maths en terminal. Voici quelques suggestions de formations que tu pourrais envisager :

### **Formations en Sciences et Ingénierie :**
1. **Licence en Sciences de la Vie et de la Terre (SVT) :**
   - **Description :** Cette licence te permet d'acquérir des connaissances en biologie, écologie, et géologie. Elle est accessible même sans spécialité maths.
   - **Parcours possibles :** Biologie, Écologie, Géologie, Environnement.
   - **Débouchés :** Chercheur, Ingénieur en environnement, Biologiste, Écologue.

2. **Licence en Biologie :**
   - **Description :** Une licence en biologie te permettra de te spécialiser dans divers domaines comme la génétique, la microbiologie, ou la biotechnologie.
   - **Parcours possibles :** Génétique, Microbiologie

In [26]:
# Test du chatbot avec une seule question utilisateur

user_input = "Je suis en terminal spécialité philosophie et littérature, mais en fait je préfère les sciences. Quelles études scientifiques pourrais-je faire malgré le fait que je n'ai pas de spé maths en terminal ?"
result = generate_response(user_input)

print("\n **Réponse du chatbot :**")
print(result["response"])


 **Réponse du chatbot :**
### Domaines identifiés :
- Sciences et Ingénierie

### Intention détectée :
- Formations

---

### 💡 Recommandation de Crystal Bot :

Je suis **Crystal Bot**, ton conseiller d'orientation professionnelle. 🔮✨

Mon objectif est de **t’aider à voir plus clair** dans ton avenir en te proposant des pistes adaptées et personnalisées.

Tu es actuellement en terminal avec des spécialités en philosophie et littérature, mais ton cœur penche vers les sciences. Ne t'inquiète pas, il existe de nombreuses voies pour te permettre de te réorienter vers des études scientifiques, même sans avoir choisi la spécialité maths en terminal.

### Formations Scientifiques Accessibles

1. **Licence de Sciences** :
   - **Licence en Sciences de la Vie** : Si tu es passionné par la biologie, la chimie ou la physique, cette licence te permettra de découvrir ces domaines en profondeur. Tu pourras ensuite te spécialiser en master.
   - **Licence en Sciences de la Terre et de l'Univers** : 

In [16]:
# Test du chatbot avec une seule question utilisateur

user_input = "Je suis en terminal spécialité philosophie, mais en fait je préfère les sciences. Quelles études scientifiques pourrais-je faire malgré le fait que je n'ai pas de spé maths en terminal ?"
result = generate_response(user_input)

    
with open("reponse_chatbot.txt", "w", encoding="utf-8") as f:
    f.write(result["response"])

print("La réponse complète a été enregistrée dans 'reponse_chatbot.txt'.")


La réponse complète a été enregistrée dans 'reponse_chatbot.txt'.


In [28]:
# Test du chatbot avec une seule question utilisateur

user_input = "J'ai besoin d'aide sur parcoursup, aide moi ?"
result = generate_response(user_input)

    
with open("reponse_chatbot.txt", "w", encoding="utf-8") as f:
    f.write(result["response"])

print("La réponse complète a été enregistrée dans 'reponse_chatbot.txt'.")

La réponse complète a été enregistrée dans 'reponse_chatbot.txt'.


In [30]:
# Test du chatbot avec une seule question utilisateur

user_input = "Je viens d'être diplômé d'un bac+3 en informatique mais je me lasse vite de faire du code toute la journée, je suis quelqu'un de sociable, peux tu me conseiller sur mon avenir ?"
result = generate_response(user_input)

    
with open("reponse_chatbot.txt", "w", encoding="utf-8") as f:
    f.write(result["response"])

print("La réponse complète a été enregistrée dans 'reponse_chatbot.txt'.")

La réponse complète a été enregistrée dans 'reponse_chatbot.txt'.


In [31]:
# Test du chatbot avec une seule question utilisateur

user_input = "J'ai une personnalité de médiateur sur mbti, j'aimerais travailler dans un métier technique mais je ne sais pas si ca pourrait me correspondre, qu'en penses tu ?"
result = generate_response(user_input)

    
with open("reponse_chatbot.txt", "w", encoding="utf-8") as f:
    f.write(result["response"])

print("La réponse complète a été enregistrée dans 'reponse_chatbot.txt'.")

La réponse complète a été enregistrée dans 'reponse_chatbot.txt'.


In [13]:
# Test du chatbot avec une seule question utilisateur

user_input = " J'hésite entre plusieurs métiers dans le droit je pense qu'en savoir plus sur les salaires pourrais m'aider ?"
result = generate_response(user_input)

    
with open("reponse_chatbot.txt", "w", encoding="utf-8") as f:
    f.write(result["response"])

print("La réponse complète a été enregistrée dans 'reponse_chatbot.txt'.")

La réponse complète a été enregistrée dans 'reponse_chatbot.txt'.


In [25]:
# Test du chatbot avec une seule question utilisateur

user_input = " Je suis en terminal option bio et physique, je ne sais pas vers quelles formation m'orienter j'ai des notes moyennes, quelles formation n'est pas trop difficile, vers quel métier ca oriente ?"
result = generate_response(user_input)

    
with open("reponse_chatbot.txt", "w", encoding="utf-8") as f:
    f.write(result["response"])

print("La réponse complète a été enregistrée dans 'reponse_chatbot.txt'.")

La réponse complète a été enregistrée dans 'reponse_chatbot.txt'.


In [49]:
# Test du chatbot avec une seule question utilisateur

user_input = " Que peux tu me dire pour trouver la formation post bac idéale?"
result = generate_response(user_input)

    
print("\n **Réponse du chatbot :**")
print(result["response"])


 **Réponse du chatbot :**
### Intention détectée :
- Formations

### Domaines identifiés :
- Autre

---

Bonjour ! 😊

Trouver la formation post-bac idéale est une étape cruciale pour ton avenir professionnel. Voici quelques conseils et alternatives pour t'aider à faire le meilleur choix :

### 1. **Auto-évaluation et Orientation**
   - **Réfléchis à tes intérêts et compétences** : Quelles matières aimes-tu le plus ? Quels sont tes points forts ?
   - **Passe des tests d’orientation** : Des plateformes comme Onisep ou Studyrama proposent des tests pour t'aider à identifier les formations qui correspondent à ton profil.

### 2. **Explorer les Options de Formation**
   - **BTS et DUT** : Ces formations professionnalisantes de deux ans sont idéales si tu préfères une approche plus pratique et directe. Elles te permettent d'acquérir des compétences techniques et de te spécialiser rapidement dans un domaine.
   - **Licence et Master** : Les licences (bac+3) et les masters (bac+5) sont plus 

In [50]:
# Test du chatbot avec une seule question utilisateur

user_input = " Que peux tu me dire pour trouver la formation post bac idéale?"
result = generate_response(user_input)

    
print("\n **Réponse du chatbot :**")
print(result["response"])


 **Réponse du chatbot :**
### Intention détectée :
- Formations

### Domaines identifiés :
- Autre

---

Bonjour ! 😊

Trouver la formation post-bac idéale est une étape cruciale pour ton avenir professionnel. Voici quelques conseils et alternatives pour t'aider à faire le meilleur choix :

### 1. **Auto-évaluation et Orientation**
   - **Réfléchis à tes intérêts et compétences** : Quelles matières aimes-tu le plus ? Quels sont tes points forts ?
   - **Passe des tests d’orientation** : Des plateformes comme Onisep ou Studyrama proposent des tests pour t'aider à identifier les formations qui correspondent à ton profil.

### 2. **Explorer les Options de Formation**
   - **BTS et DUT** : Ces formations professionnalisantes de deux ans sont idéales si tu préfères une approche plus pratique et directe. Elles te permettent d'acquérir des compétences techniques et de te spécialiser rapidement dans un domaine.
   - **Licence et Master** : Les licences (bac+3) et les masters (bac+5) sont plus 

In [21]:
# Test après modification 

user_input = "Je suis en terminal spécialité philosophie, mais en fait je préfère les sciences. Quelles études scientifiques pourrais-je faire malgré le fait que je n'ai pas de spé maths en terminal ?"
result = generate_response(user_input)

print("\n **Réponse du chatbot :**")
print(result["response"])



 **Réponse du chatbot :**
### Bonjour !

Je comprends que tu es en terminal avec une spécialité en philosophie, mais que tu te sens plus attiré par les sciences. Pas de souci, il existe de nombreuses options pour te permettre de te réorienter vers des études scientifiques, même sans avoir choisi la spécialité maths. Voici quelques pistes pour t'aider à y voir plus clair :

### Intention détectée :
- **Formations**

### Domaines identifiés :
- Sciences et Ingénierie

### Recommandations de Crystal Bot :

#### Formations Pertinentes

1. **Licence en Sciences de la Vie et de la Terre (SVT)** :
   - **Description** : Cette licence te permet d’explorer divers aspects des sciences naturelles, de la biologie à la géologie. Elle est idéale pour ceux qui sont intéressés par la recherche scientifique et les études environnementales.
   - **Alternatives** : Si tu es intéressé par les sciences de la vie, tu peux également envisager une licence en biologie ou en écologie.

2. **BTS Biotechnologies

In [16]:
# Test du chatbot avec une seule question utilisateur

user_input = " Que peux tu me dire pour trouver la formation post bac idéale?"
result = generate_response(user_input)

    
print("\n **Réponse du chatbot :**")
print(result["response"])


 **Réponse du chatbot :**
Bonjour ! 😊

Je suis **Crystal Bot**, ton conseiller d'orientation professionnelle. 🔮✨

Mon objectif est de **t’aider à voir plus clair** dans ton avenir en te proposant des pistes adaptées et personnalisées.

Pour trouver la formation post-bac idéale, voici quelques conseils et suggestions :

### **1. **Identifie tes centres d'intérêt et tes compétences**
Avant de te lancer dans une formation, il est essentiel de bien comprendre ce qui te passionne et ce dans quoi tu es doué. Fais une liste de tes intérêts, même les plus inattendus, et voit comment ils pourraient se traduire en une formation ou un métier.

### **2. **Explore différentes options de formation**
Il existe une multitude de formations possibles, allant des filières traditionnelles aux parcours plus atypiques. Voici quelques options à considérer :

- **BTS (Brevet de Technicien Supérieur)** : Idéal pour une formation rapide et spécialisée dans un domaine technique. Par exemple, le BTS Design Graph